# Test Imperfect Lorenz Model 

In [1]:
from pathlib import Path
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

# Add src folder to Python path
sys.path.append(str(Path("../src").resolve()))

from data_generation import generate_data
from lstm import WindowedTimeSeries, train_model, predict_one_step, predict_autoregressive
from ode_model import LorenzModel

In [2]:
data = generate_data(10000, noise_level=0.05)

In [3]:
model = LorenzModel()

In [4]:
data.shape

(3, 10000)

In [5]:
preds = model.predict(y0=data.T, t_step=0.05)

In [7]:
time_step = 0.05


for c in range(10,21):
  model.update_params(c=c)
  preds = model.predict(y0=data.T, t_step=time_step*10)
  true = data.T

  preds = preds[:-10, :]
  true = true[10: , :]
  mse = np.mean((preds - true) ** 2)
  print(f"c: {c}, MSE: {mse}")


c: 10, MSE: 377.2689796705706
c: 11, MSE: 145.47339318143253
c: 12, MSE: 38.456655776951294
c: 13, MSE: 22.75846038514155
c: 14, MSE: 20.37375790773256
c: 15, MSE: 19.34918539605423
c: 16, MSE: 18.810129718748968
c: 17, MSE: 18.496293806482225
c: 18, MSE: 18.302154639774596
c: 19, MSE: 18.17621198455573
c: 20, MSE: 18.09261245733633


C = 13 matches MSE of LSTM for prediction of step = 10